# Collateral Data
#### Airport names and IATA codes from the internet
Wikipedia has a list of US airports that we can use. Note that this code may break whenever the wikipedia page changes. In production consider more reliable data sources.

In [21]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import pandas_gbq as pgbq

In [22]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [23]:
url='https://en.wikipedia.org/wiki/List_of_airports_in_the_United_States'

In [24]:
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [25]:
table = html_soup.find_all('table')[2]

In [26]:
all_rows = table.find_all('tr')

In [27]:
import re
data = [ 
    [
    str(re.sub(r'[^\x00-\x7f]',r' ', r.find_all('td')[4].text.strip())),     
    str(r.find_all('td')[2].text.strip())
    ] 
    for r in all_rows 
        if r.find_all('th') == [] and 
        str(r.find_all('td')[2].text.strip()) != '' and
        r.attrs.get('style') == None]

In [28]:
len(data)

381

In [29]:
#df = pd.DataFrame(data, columns=["Name", "IATA"])
#df.to_gbq(project_id='going-tfx', destination_table='examples.AIRPORTS')

---
Hmm. Pretty disappointing. Only 381. Quickaid.com has more.

## Quickaid.com

More airports are available from [quickaid.com](http://quickaid.com/usa-airport-codes/#USM)

In [30]:
url='http://quickaid.com/usa-airport-codes/'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [31]:
def parse(soup):
    for ul in soup.find_all('ul')[2:25]:
        for li in ul.find_all('li'):
            text=li.text.encode('ascii', 'ignore')
            comma = text.find(',')
            yield[ text[:-5].strip(), text[comma+2:comma+4], text[-4:-1]]

res = list(parse(html_soup))
len(res)

728

In [32]:
airports_pd = pd.DataFrame(parse(html_soup), columns=["Name", "State", "IATA"])

In [33]:
airports_pd[:10]

,Name,State,IATA
0,"Aberdeen, SD",SD,ABR
1,"Abilene, TX",TX,ABI
2,"Adak Island, AK",AK,ADK
3,"Akiachak, AK",AK,KKI
4,"Akiak, AK",AK,AKI
5,"Akron/Canton, OH",OH,CAK
6,"Akuton, AK",AK,KQA
7,"Alakanuk, AK",AK,AUK
8,"Alamogordo, NM",NM,ALM
9,"Alamosa, CO",CO,ALS


---
These Washington DC and New London records fail to parse the state correctly because they have no comma in them. So we correct them manually.

In [34]:
[ airports_pd.iloc[i] for i in [462,699,700,701]]

[Name     New London/Groton
 State                   ew
 IATA                   GON
 Name: 462, dtype: object, Name     Washington DC  All airports
 State                             as
 IATA                             WAS
 Name: 699, dtype: object, Name     Washington DC  Dulles
 State                       as
 IATA                       IAD
 Name: 700, dtype: object, Name     Washington DC  National
 State                         as
 IATA                         DCA
 Name: 701, dtype: object]

In [35]:
airports_pd.iloc[462]['State']='CT'
airports_pd.iloc[699]['State']='DC'
airports_pd.iloc[700]['State']='DC'
airports_pd.iloc[701]['State']='DC'

---
Some IATA codes are duplicates. We drop all but the first of each group of rows for a particular IATA, thereby risking to choose a name that might have expired. Can't help it.

In [36]:
len(airports_pd)

728

In [37]:
airports_pd = airports_pd.drop_duplicates('IATA')

In [38]:
len(airports_pd)

676

We'll write the records to BQ already to make use of its powerful analytics capabilities. Would've been a bit harder with pandas.

In [39]:
airports_pd.to_gbq(project_id='going-tfx', destination_table='examples.AIRPORTS', if_exists='replace')

---
There are 348 distinct airports reported of in the flight data table

In [42]:
%%bigquery
with unique_airports as (
  select 
    distinct departure_airport, departure_lat, departure_lon
  from `bigquery-samples.airline_ontime_data.flights` 
  union distinct
  select 
    distinct arrival_airport, arrival_lat, arrival_lon
  from `bigquery-samples.airline_ontime_data.flights` 
)
select count(*) from unique_airports 

,f0_
0,348


---
Still, 21 of them are not recorded in our list of airports

In [43]:
%%bigquery
with unique_airports as (
  select 
    distinct departure_airport as iata, departure_lat as lat, departure_lon as lon
  from `bigquery-samples.airline_ontime_data.flights` 
  union distinct
  select 
    distinct arrival_airport as iata, arrival_lat as lat, arrival_lon as lon
  from `bigquery-samples.airline_ontime_data.flights` 
)
select u.iata from 
    unique_airports u left outer join `going-tfx.examples.AIRPORTS` a on
    u.iata = a.iata
where a.iata is null

,iata
0,BQN
1,STT
2,HOB
3,MAZ
4,ISO
5,MMH
6,EWN
7,EFD
8,LYH
9,GUM


I know it wasn't necessary, but I couldn't help but search Wikipedia until I had them all!

In [44]:
missing = [
    ['Rafael Hernandez Airport', 'Pu', 'BQN' ],
    ['Cyril E. King Airport','VI','STT'],
    ['Lea County Regional Airport','NM','HOB'],
    ['Eugenio Maria de Hostos Airport','Pu','MAZ'],
    ['Kinston Regional Jetport','NC','ISO'],
    ['Mammoth Yosemite Airport','CA','MMH'],
    ['Coastal Carolina Regional Airport','NC','EWN'],
    ['Ellington Airport','TX','EFD'],
    ['Lynchburg Regional Airport','VA','LYH'],
    ['Antonio B. Won Pat International Airport (Guam)','GU','GUM'],
    ['Provo Municipal Airport','UT','PVU'],
    ['Southwest Georgia Regional Airport','GA','ABY'],
    ['Robert Gray Army Airfield','TX','GRK'],
    ['Florida Keys Marathon Airport','FL','MTH'],
    ['Columbus Air Force Base','MS','CBM'],
    ['Henry E. Rohlsen Airport','VI','STX'],
    ['Ogden-Hinckley Airport','UT','OGD'],
    ['Charles B. Wheeler Downtown Airport','MO','MKC'],
    ['Yakutat Airport','AK','YAK'],
    ['Yakima Air Terminal','WA','YKM'],
    ['Palmdale Regional Airport','CA','PMD']
]

In [45]:
import numpy as np
name, state, iata = np.transpose(missing) 

In [46]:
additional = pd.DataFrame({'Name': name, 'State': state, 'IATA': iata})

In [47]:
finally_all = pd.concat([airports_pd, additional], sort=True)

In [48]:
finally_all[finally_all['State']=='Pu']

,IATA,Name,State
531,PSE,"Ponce, Puerto Rico",Pu
595,SJU,"San Juan, Puerto Rico",Pu
0,BQN,Rafael Hernandez Airport,Pu
3,MAZ,Eugenio Maria de Hostos Airport,Pu


Puerto Rico should actually be 'PR'. We'll fix that after another round trip to BQ, since iloc doesn't work as expected after the concatenation

In [49]:
finally_all.to_gbq(project_id='going-tfx', destination_table='examples.AIRPORTS', if_exists='replace')

In [50]:
%%bigquery finally_all
select * from `examples.AIRPORTS`

,IATA,Name,State
0,ADK,"Adak Island, AK",AK
1,KKI,"Akiachak, AK",AK
2,AKI,"Akiak, AK",AK
3,KQA,"Akuton, AK",AK
4,AUK,"Alakanuk, AK",AK
5,WKK,"Aleknagik, AK",AK
6,AET,"Allakaket, AK",AK
7,ABL,"Ambler, AK",AK
8,AKP,"Anaktueuk, AK",AK
9,ANC,"Anchorage, AK",AK


In [51]:
finally_all[finally_all['State']=='Pu']

,IATA,Name,State
583,PSE,"Ponce, Puerto Rico",Pu
584,SJU,"San Juan, Puerto Rico",Pu
585,BQN,Rafael Hernandez Airport,Pu
586,MAZ,Eugenio Maria de Hostos Airport,Pu


In [52]:
finally_all.iloc[583]['State']='PR'
finally_all.iloc[584]['State']='PR'
finally_all.iloc[585]['State']='PR'
finally_all.iloc[586]['State']='PR'

In [53]:
finally_all[finally_all['State']=='PR']

,IATA,Name,State
583,PSE,"Ponce, Puerto Rico",PR
584,SJU,"San Juan, Puerto Rico",PR
585,BQN,Rafael Hernandez Airport,PR
586,MAZ,Eugenio Maria de Hostos Airport,PR


In [54]:
finally_all.to_gbq(project_id='going-tfx', destination_table='examples.AIRPORTS', if_exists='replace')

In [56]:
%%bigquery airports_geo
with unique_airports as (
  select 
    distinct departure_airport as airport, departure_lat as lat, departure_lon as lon
  from `bigquery-samples.airline_ontime_data.flights` 
  union distinct
  select 
    distinct arrival_airport as airport, arrival_lat as lat , arrival_lon as lon
  from `bigquery-samples.airline_ontime_data.flights` 
)
select 
  a.name, u.airport, a.state, u.lat, u.lon
from 
  unique_airports u join
  `going-tfx.examples.AIRPORTS` a
on 
  u.airport = a.iata

,name,airport,state,lat,lon
0,"Adak Island, AK",ADK,AK,51.87,-176.64
1,"Anchorage, AK",ANC,AK,61.17,-149.99
2,"Barrow, AK",BRW,AK,71.28,-156.76
3,"Bethel, AK",BET,AK,60.77,-161.83
4,"Cordova, AK",CDV,AK,60.49,-145.47
5,"Dillingham, AK",DLG,AK,59.04,-158.50
6,"Dutch Harbor, AK",DUT,AK,53.90,-166.54
7,"Fairbanks, AK",FAI,AK,64.81,-147.85
8,"Gustavus, AK",GST,AK,58.42,-135.70
9,"Juneau, AK",JNU,AK,58.35,-134.57


In [57]:
airports_geo.to_gbq(project_id='going-tfx', destination_table='examples.AIRPORTS_GEO', if_exists='replace')